In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 下载MNIST数据集并生成DataSet对象
# 使用OneHot编码处理标记
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


使用便捷内置代价函数：
* `tf.nn.softmax_cross_entropy_with_logits_v2`
* `tf.keras.losses.categorical_crossentropy`

In [3]:
with tf.Graph().as_default() as g:
    # 输入、标记占位符
    inputs = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    
    # 创建128个隐藏层神经元参数
    hidden_weight = tf.Variable(tf.random_normal([784, 128]), name='hidden_weight')
    hidden_bias = tf.Variable(tf.zeros([128, ]), name='hidden_bias')
    # 隐藏层前向传播
    hidden_output = tf.nn.relu(tf.matmul(inputs, hidden_weight) + hidden_bias)
    
    
    # 创建输出层10个神经元参数
    output_weight = tf.Variable(tf.random_normal([128, 10], name='output_weight'))
    output_bias = tf.Variable(tf.zeros([10, ]), name='output_bias')
    # 输出层前向传播
    logits = tf.matmul(hidden_output, output_weight) + output_bias
    output = tf.nn.softmax(logits)
    
    
    # 代价函数
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=labels, axis=1))
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

Instructions for updating:
Colocations handled automatically by placer.
step     0, loss 125.6319, acc 0.0837
step   500, loss 3.3931, acc 0.7494
step  1000, loss 2.6974, acc 0.8095
step  1500, loss 0.4353, acc 0.8312
step  2000, loss 1.2433, acc 0.8424
step  2500, loss 3.8981, acc 0.8558
step  3000, loss 1.9917, acc 0.8610
step  3500, loss 5.1736, acc 0.8661
step  4000, loss 2.4020, acc 0.8693
step  4500, loss 3.5425, acc 0.8749
step  5000, loss 1.1250, acc 0.8754
step  5500, loss 1.3511, acc 0.8767
step  6000, loss 2.5899, acc 0.8802
step  6500, loss 1.5756, acc 0.8835
step  7000, loss 0.9136, acc 0.8867
step  7500, loss 1.9575, acc 0.8884
step  8000, loss 1.7793, acc 0.8837
step  8500, loss 1.1245, acc 0.8886
step  9000, loss 0.1695, acc 0.8921
step  9500, loss 0.8470, acc 0.8873
step 10000, loss 1.7650, acc 0.8914
step 10500, loss 0.6030, acc 0.8909
step 11000, loss 0.2352, acc 0.8973
step 11500, loss 0.0262, acc 0.8952
step 12000, loss 1.5102, acc 0.8943
step 12500, loss 1.6592, a

使用便捷的高级完成神经网络层的创建：
* `tf.keras.layers.Dense`

In [4]:
with tf.Graph().as_default() as g:
    # 输入、标记占位符
    inputs = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    # 隐藏层与输出层
    hidden_output = tf.keras.layers.Dense(
        128, activation=tf.nn.relu)(inputs)
    
    logits = tf.keras.layers.Dense(
        10, activation=None)(hidden_output)
    output = tf.nn.softmax(logits)
    
    # 代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=labels, axis=1))
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.5095, acc 0.1047
step   500, loss 0.4933, acc 0.8596
step  1000, loss 0.3141, acc 0.8896
step  1500, loss 0.1737, acc 0.8957
step  2000, loss 0.5406, acc 0.9072
step  2500, loss 0.3244, acc 0.9099
step  3000, loss 0.3735, acc 0.9162
step  3500, loss 0.2050, acc 0.9169
step  4000, loss 0.3043, acc 0.9188
step  4500, loss 0.3187, acc 0.9225
step  5000, loss 0.2003, acc 0.9245
step  5500, loss 0.3578, acc 0.9298
step  6000, loss 0.1006, acc 0.9292
step  6500, loss 0.3964, acc 0.9277
step  7000, loss 0.1702, acc 0.9335
step  7500, loss 0.0786, acc 0.9309
step  8000, loss 0.0981, acc 0.9343
step  8500, loss 0.0831, acc 0.9338
step  9000, loss 0.2233, acc 0.9361
step  9500, loss 0.3473, acc 0.9341
step 10000, loss 0.1391, acc 0.9377
step 10500, loss 0.1955, acc 0.9402
step 11000, loss 0.1924, acc 0.9400
step 11500, loss 0.2447, acc 0.9417
step 12000, loss 0.1134, acc 0.9415
step 12500, loss 0.3065, acc 0.9424
step 13000, loss 0.5117, acc 0.9433
step 13500, loss 0.3654, acc